# MNIST dataset for hand written digits
### important remarks
- The data has 60000 image
- Eachh category has around the same number of data points

In [2]:
from fastai.vision.all import *
import numpy as np
import os

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
#function convert the image files in a certain path into a list of tensors and save them in a dictionary
def data_tensors(path):
    files = os.listdir(path)
    Urls = (path).ls()
    data_set = {}
    for i, j in zip(files, Urls):
        data_set[i] = [tensor(Image.open(im)) for im in j.ls()]
    return data_set


In [5]:
data_train = data_tensors(path/"training")
data_test = data_tensors(path/"testing")

In [7]:
f = lambda x: torch.stack(x)

data_train = {i : f(j) for i, j in data_train.items()}
data_test = {i : f(j) for i, j in data_test.items()}

In [10]:
x_train = torch.cat(list(data_train.values())).view(-1, 28*28).float()/255
y_train = torch.cat( [tensor([int(i)]*len(data[i])) for i in list(data.keys())])

x_test = torch.cat(list(data_test.values())).view(-1, 28*28).float()/255
y_test = torch.cat( [tensor([int(i)]*len(data_test[i])) for i in list(data_test.keys())])

In [118]:
#simple sequantial neural netwrok model
neural_net = nn.Sequential(
nn.Linear(28*28, 550),
    nn.ReLU(),
    nn.Linear(550, 350),
    nn.ReLU(),
    nn.Linear(350, 200),
nn.ReLU(),
nn.Linear(200, 100),
nn.ReLU(),
nn.Linear(100,60),
nn.ReLU(),
nn.Linear(60, 30),
nn.ReLU(),
nn.Linear(30, 10))

In [13]:
des_t = list(zip(x_train, y_train))
des_val = list(zip(x_test, y_test))

In [110]:
# using fastai data loaders for batching the training data
dl_t = DataLoader(des_t, batch_size= 125, shuffle =  True)

In [22]:
# loss function used for otimization
#using the entropy loss function from pytorch

def loss_function(preds, yb):
    loss = F.cross_entropy(preds,yb.squeeze())
    return loss
    

In [99]:
# optimizer class to do the optimization steps
class optmizer:
    def __init__ (self, parameters, lr):
        self.parameters = list(parameters)
        self.lr = lr
    def step(self, *args, **kwargs):
        for p in self.parameters:
            p.data -= p.grad.data * self.lr
            
    def set_zero(self,*args, **kwargs ):
        for p in self.parameters:
            p.grad = None

In [ ]:
# calculate the accurecy the predections 
def _accurecy(pred, targets):
    preds = torch.argmax(pred.softmax(1),1)
    return (preds == targets).float().mean()
    

In [102]:
#stocastic gredient descent function
def sgd(xb , yb , model):
    preds = model(xb)
    loss = loss_function(preds, yb)
    loss.backward()

In [103]:
#training for one epochs
def train_epochs(dl, model , prt = False):
    
    for xb,yb in dl:
        sgd(xb,yb, model)
        opt.step()
        opt.set_zero()
        if prt:
            print("batch_accurecy = {:.3f}".format(_accurecy(model(xb) , yb)))

In [107]:
def model_training(dl , model, epochs):
    for i in range(epochs):
        train_epochs(dl, model)
        print("epoch {} accuracy = {:.3f}".format(i ,_accurecy(model(x_test), y_test)))
      

In [119]:
opt= optmizer(neural_net.parameters(), .1)
model_training(dl_t, neural_net, 10)

epoch 0 accuracy = 0.207
epoch 1 accuracy = 0.534
epoch 2 accuracy = 0.937
epoch 3 accuracy = 0.949
epoch 4 accuracy = 0.960
epoch 5 accuracy = 0.966
epoch 6 accuracy = 0.969
epoch 7 accuracy = 0.972
epoch 8 accuracy = 0.974
epoch 9 accuracy = 0.973
